# Swiss City Clustering By Venue Types

In [37]:
# imports
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from math import ceil

#! pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors
#import matplotlib.pyplot as plt
import branca
#import branca.colormap

# import k-means from clustering stage
from sklearn.cluster import KMeans

#! pip install folium
import folium # map rendering library

# Foursquare credentials and version are in separate config file that is not in the git repository so my credentials are not published.
import foursquare_config

print('Libraries imported.')

Libraries imported.


## Swiss City Data Analysis

Get table of swiss cities, clean it and sort it by population.

In [2]:
cities = pd.DataFrame(pd.read_html('https://en.wikipedia.org/wiki/List_of_cities_in_Switzerland', header=1)[0])
cities.drop(cities.columns[1], axis=1, inplace=True)
cities.columns = ['Town', 'District', 'Canton', 'Town Population', 'Agglomeration Population', 'Agglomeration']
cities

,Town,District,Canton,Town Population,Agglomeration Population,Agglomeration
0,Aarau,Aarau,AG,21506,76636.0,Aarau
1,Aarberg,Aarberg,BE,4628,NaN,-
2,Aarburg,Zofingen,AG,8197,98535.0,Olten–Zofingen
3,Adliswil,Horgen,ZH,18769,1334269.0,Zurich
4,Aesch (BL)[note 1],Arlesheim,BL,10440,541011.0,Basel (CH)
5,Affoltern am Albis[note 2],Affoltern,ZH,12229,1334269.0,Zurich
6,Agno[note 2],Lugano,TI,4445,151037.0,Lugano (CH)
7,Aigle,Aigle,VD,10119,NaN,-
8,Allschwil[note 2],Arlesheim,BL,21248,541011.0,Basel (CH)
9,Altdorf (UR)[note 2],-,UR,9401,31734.0,Altdorf (UR)


In [3]:
cities.dtypes

Town                         object
District                     object
Canton                       object
Town Population              object
Agglomeration Population    float64
Agglomeration                object
dtype: object

Change Town Population to numeric type and sort by Town Population.

In [4]:
cities['Town Population'] = pd.to_numeric(cities['Town Population'], errors='coerce')
cities = cities[~cities['Town Population'].isnull()]
cities.sort_values('Town Population', ascending=False, inplace=True)
cities.reset_index(drop=True, inplace=True)
print(cities.dtypes)
cities.head()

Town                         object
District                     object
Canton                       object
Town Population             float64
Agglomeration Population    float64
Agglomeration                object
dtype: object


<ipython-input-4-38166949d52c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities.sort_values('Town Population', ascending=False, inplace=True)


,Town,District,Canton,Town Population,Agglomeration Population,Agglomeration
0,Zürich,Zurich,ZH,415367.0,1334269.0,Zurich
1,Geneva,-,GE,201818.0,579227.0,Genève (CH)
2,Basel,-,BS,177654.0,541011.0,Basel (CH)
3,Lausanne,Lausanne,VD,139111.0,409295.0,Lausanne
4,Bern,Bern-Mittelland,BE,133883.0,410894.0,Bern


Remove [note 1] and [note 2] from town names.

In [5]:
cities['Town'] = cities['Town'].str.replace(r'(\[note [12]\])|\*$', '', regex=True)

<ipython-input-5-96364a9c203f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities['Town'] = cities['Town'].str.replace(r'(\[note [12]\])|\*$', '', regex=True)


Remove cities with a population under 10000.

In [6]:
min_population_to_consider = 10000
cities = cities[cities['Town Population'] >= min_population_to_consider]
cities

,Town,District,Canton,Town Population,Agglomeration Population,Agglomeration
0,Zürich,Zurich,ZH,415367.0,1334269.0,Zurich
1,Geneva,-,GE,201818.0,579227.0,Genève (CH)
2,Basel,-,BS,177654.0,541011.0,Basel (CH)
3,Lausanne,Lausanne,VD,139111.0,409295.0,Lausanne
4,Bern,Bern-Mittelland,BE,133883.0,410894.0,Bern
5,Winterthur,Winterthur,ZH,111851.0,138252.0,Winterthur
6,Lucerne,Lucerne,LU,81691.0,226091.0,Lucerne
7,St. Gallen,St. Gallen,SG,75833.0,165860.0,St. Gallen
8,Lugano,Lugano,TI,63185.0,151037.0,Lugano (CH)
9,Biel/Bienne,Biel/Bienne,BE,55159.0,104542.0,Biel/Bienne


Define function to get coordinates and try it out.

In [7]:
try_town = 'Chur'
geoloc = Nominatim(user_agent="swiss_explorer")

def get_lat_lon_from_address(address, geolocator=geoloc):
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

def get_coords_from_town(town_name, country='Switzerland', geolocator=geoloc):
    address = town_name + ', ' + country
    return get_lat_lon_from_address(address, geolocator)

latitude, longitude = get_coords_from_town(try_town)
print('The geograpical coordinate of {} are {}, {}.'.format(try_town, latitude, longitude))

The geograpical coordinate of Chur are 46.855515, 9.5254066.


Get the coordinates for all cities.

In [8]:
lat_lon = cities['Town'].apply(get_coords_from_town)

In [9]:
print(lat_lon[:10])
cities['Latitude'] = lat_lon.map(lambda x: x[0])
cities['Longitude'] = lat_lon.map(lambda x: x[1])
print(cities.dtypes)
display(cities.head())

0    (47.3723941, 8.5423328)
1    (46.2017559, 6.1466014)
2    (47.5581077, 7.5878261)
3    (46.5218269, 6.6327025)
4    (46.9482713, 7.4514512)
5    (47.4991723, 8.7291498)
6    (47.0505452, 8.3054682)
7    (47.4250593, 9.3765878)
8    (46.0050102, 8.9520281)
9    (47.1402077, 7.2439029)
Name: Town, dtype: object
Town                         object
District                     object
Canton                       object
Town Population             float64
Agglomeration Population    float64
Agglomeration                object
Latitude                    float64
Longitude                   float64
dtype: object


,Town,District,Canton,Town Population,Agglomeration Population,Agglomeration,Latitude,Longitude
0,Zürich,Zurich,ZH,415367.0,1334269.0,Zurich,47.372394,8.542333
1,Geneva,-,GE,201818.0,579227.0,Genève (CH),46.201756,6.146601
2,Basel,-,BS,177654.0,541011.0,Basel (CH),47.558108,7.587826
3,Lausanne,Lausanne,VD,139111.0,409295.0,Lausanne,46.521827,6.632702
4,Bern,Bern-Mittelland,BE,133883.0,410894.0,Bern,46.948271,7.451451


Plot location of cities and their population on a map. The area of the marker points is proportional to the population and the marker points are color coded by population.

In [10]:
def population_to_marker_radius(population, factor=3e-2):
    return factor*np.sqrt(population)

color_min_populations = np.array([10000, 25000, 50000, 100000, 250000, 500000])
population_colors = ['magenta', 'blue', 'green', 'orange', 'yellow']
population_divisor = 1000;

max_population = cities['Town Population'].max()

population_colormap =  branca.colormap.StepColormap(population_colors, color_min_populations/population_divisor, vmin=color_min_populations[0]/population_divisor, vmax=max_population/population_divisor)
population_colormap.caption = 'Town Population in thousands (year 2018)'

#def population_colormap(population, min_vals=color_min_populations, colors=population_colors):
#    col_index = bisect.bisect_left(min_vals, population) - 1
#    assert 0 <= col_index < len(colors)
#    return colors[col_index]

In [11]:
# central coordinates of switzerland
swiss_lat_lon_coords = (46.8182, 8.2275)
# create map of Manhattan using latitude and longitude values
map_switzerland = folium.Map(location=swiss_lat_lon_coords, zoom_start=8)

# add markers to map
for lat, lng, label, population in zip(cities['Latitude'], cities['Longitude'], cities['Town'], cities['Town Population']):
    # deal with Umlaut characters
    label = label.encode('ascii', 'xmlcharrefreplace').decode('utf-8')
    label = folium.Popup(label + ', Population: ' + str(int(population)), parse_html=False)
    folium.CircleMarker(
        [lat, lng],
        radius=population_to_marker_radius(population),
        popup=label,
        color=population_colormap(population/population_divisor),
        fill=True,
        fill_color=population_colormap(population/population_divisor),
        fill_opacity=0.7,
        parse_html=False).add_to(map_switzerland)  
    
population_colormap.add_to(map_switzerland)    
map_switzerland

## Venue Analysis

Get the venues for around the city centers in a radius of 1000 m.

In [93]:
def getNearbyVenues(names, latitudes, longitudes, radius=None, limit=100, max_tries=10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        base_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
            foursquare_config.CLIENT_ID, 
            foursquare_config.CLIENT_SECRET, 
            foursquare_config.VERSION, 
            lat, 
            lng
        )
        if radius is not None:
            base_url = base_url + '&radius={}'.format(radius)
        if limit is not None:
            base_url = base_url + '&limit={}'.format(limit)
            
        # make the GET request
        isResultOk = False
        for try_count in range(max_tries):
            if isResultOk:
                break
            base_results_json = requests.get(base_url).json()
            try:
                n_total_results = base_results_json['response']['totalResults']
            except:
                print('Problem occurred in initial search. Retry.')
                print(base_results_json['response'])
                continue
            isResultOk = True
        print(str(n_total_results) + ' results')
        
        for iGet in range(math.ceil(n_total_results/limit)):
            url = base_url + '&offset={}'.format(limit*iGet)
            isResultOk = False
            for try_count in range(max_tries):
                if isResultOk:
                    break
                results_json = requests.get(url).json()
                try:
                    results = results_json["response"]['groups'][0]['items']
                except:
                    print('Problem occurred with offset {}. Retry.'.format(limit*iGet))
                    continue
                isResultOk = True
                
            if isResultOk:
                # return only relevant information for each nearby venue
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])
            else:
                print('Skipping query')

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Town Latitude', 
                  'Town Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Test function on a familiar city

In [91]:
zvenues = getNearbyVenues(['Zürich'], ['47.3769'], ['8.5417'], radius=None, limit=50)

Zürich
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial search. Retry.
{}
Problem occurred in initial

UnboundLocalError: local variable 'n_total_results' referenced before assignment

In [81]:
print(zvenues.shape)
zvenues.head()

(182, 7)


,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zürich,47.3769,8.5417,Dachterrasse Hiltl,47.375686,8.539650,Vegetarian / Vegan Restaurant
1,Zürich,47.3769,8.5417,Grande Café & Bar,47.375479,8.543395,Bar
2,Zürich,47.3769,8.5417,Läderach,47.376537,8.539653,Chocolate Shop
3,Zürich,47.3769,8.5417,Yumi Hana Lebensmittel & Spezialitäten,47.376343,8.540386,Grocery Store
4,Zürich,47.3769,8.5417,Sprüngli,47.377401,8.540111,Candy Store


Get all venues for the table of cities.

In [83]:
venues = getNearbyVenues(cities['Town'], latitudes=cities['Latitude'], longitudes=cities['Longitude'])

Zürich
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Problem occurred in initial search. Retry.
Prob

UnboundLocalError: local variable 'n_total_results' referenced before assignment

In [95]:
print('Number of venues: ' + str(len(venues.index)))
print('Number of unique categories:' + str(venues['Venue Category'].nunique()))
print('Number of cities: ' + str(len(cities.index)))
display(venues.head())
# TODO check how many venues were found in the city with the least venues found
display(venues.tail())

Number of venues: 13589
Number of unique categories:336
Number of cities: 152


,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zürich,47.372394,8.542333,Lindenhof,47.373005,8.540883,Pedestrian Plaza
1,Zürich,47.372394,8.542333,Old Crow,47.372092,8.541024,Cocktail Bar
2,Zürich,47.372394,8.542333,Café Schober,47.371400,8.544149,Café
3,Zürich,47.372394,8.542333,Widder Bar,47.372415,8.539863,Hotel Bar
4,Zürich,47.372394,8.542333,Widder Hotel,47.372449,8.539973,Hotel


,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
13584,Steinhausen,47.197508,8.486014,ZVB Menzingen,47.178309,8.590482,Bus Station
13585,Steinhausen,47.197508,8.486014,Restaurant Bäckerei Schlüssel,47.178352,8.590663,Bakery
13586,Steinhausen,47.197508,8.486014,Restaurant & Hotel Ochsen,47.177983,8.590951,Swiss Restaurant
13587,Steinhausen,47.197508,8.486014,Reit- und Pensionsstall Hirzel,47.216847,8.595458,Stables
13588,Steinhausen,47.197508,8.486014,Schreinerei Kleiner,47.216781,8.596025,Factory


Look at categories in order of popularity.

In [107]:
venues['Venue Category'].value_counts()

Swiss Restaurant                   1029
Supermarket                         922
Hotel                               909
Restaurant                          674
Italian Restaurant                  576
Train Station                       507
Bar                                 458
Café                                432
Grocery Store                       386
Park                                271
Shopping Mall                       247
Fast Food Restaurant                241
Bakery                              240
French Restaurant                   239
Pizza Place                         203
Plaza                               186
Coffee Shop                         165
Beach                               127
Pool                                122
Gym / Fitness Center                120
Department Store                    109
Burger Joint                        105
Gym                                 103
Thai Restaurant                      99
Asian Restaurant                     93


No obvious categories that need to be merged in the top categories. Delete Town Venue because it does not make sense to include a venue for the town itself.

## City Clustering

Do one hot encoding on venue categories. Then take the average of the encodings to get a unit vector for each city that depends on the venue categories in the city. These vectors are the features for the clustering.

In [110]:
# one hot encoding.
swiss_venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
swiss_venues_onehot = pd.concat([venues['Town'], swiss_venues_onehot], axis=1)
display(swiss_venues_onehot.head())

swiss_grouped = swiss_venues_onehot.groupby('Town').mean().reset_index()
swiss_grouped.head()

,Town,C_Accessories Store,C_Airport,C_Airport Lounge,C_Airport Service,C_Airport Terminal,C_American Restaurant,C_Apres Ski Bar,C_Arcade,C_Argentinian Restaurant,C_Art Gallery,C_Art Museum,C_Arts & Crafts Store,C_Asian Restaurant,C_Athletics & Sports,C_Australian Restaurant,C_Auto Dealership,C_Auto Garage,C_Auto Workshop,C_BBQ Joint,C_Bagel Shop,C_Bakery,C_Bank,C_Bar,C_Basketball Stadium,C_Bath House,C_Bathing Area,C_Beach,C_Beach Bar,C_Bed & Breakfast,C_Beer Garden,C_Beer Store,C_Big Box Store,C_Bistro,C_Boat or Ferry,C_Bookstore,C_Border Crossing,C_Botanical Garden,C_Boutique,C_Bowling Alley,C_Bratwurst Joint,C_Brazilian Restaurant,C_Breakfast Spot,C_Brewery,C_Bridge,C_Bubble Tea Shop,C_Buffet,C_Burger Joint,C_Burrito Place,C_Bus Line,C_Bus Station,C_Bus Stop,C_Butcher,C_Cable Car,C_Cafeteria,C_Café,C_Campground,C_Canal Lock,C_Candy Store,C_Car Wash,C_Casino,C_Castle,C_Cheese Shop,C_Chinese Restaurant,C_Chocolate Shop,C_Christmas Market,C_Church,C_Circus,C_City,C_Climbing Gym,C_Clothing Store,C_Cocktail Bar,C_Coffee Shop,C_Comedy Club,C_Comfort Food Restaurant,C_Community Center,C_Concert Hall,C_Construction & Landscaping,C_Convenience Store,C_Convention Center,C_Cosmetics Shop,C_Coworking Space,C_Creperie,C_Cuban Restaurant,C_Cultural Center,C_Cupcake Shop,C_Dance Studio,C_Deli / Bodega,C_Department Store,C_Dessert Shop,C_Dim Sum Restaurant,C_Diner,C_Discount Store,C_Dive Bar,C_Doner Restaurant,C_Donut Shop,C_Drugstore,C_Eastern European Restaurant,C_Electronics Store,C_English Restaurant,C_Ethiopian Restaurant,C_Event Space,C_Exhibit,C_Factory,C_Falafel Restaurant,C_Farm,C_Farmers Market,C_Fast Food Restaurant,C_Field,C_Fish & Chips Shop,C_Fishing Spot,C_Flea Market,C_Flower Shop,C_Fondue Restaurant,C_Food,C_Food & Drink Shop,C_Food Court,C_Food Truck,C_Forest,C_Fountain,C_French Restaurant,C_Fried Chicken Joint,C_Furniture / Home Store,C_Garden,C_Garden Center,C_Gas Station,C_Gastropub,C_General College & University,C_General Entertainment,C_General Travel,C_German Restaurant,C_Gift Shop,C_Golf Course,C_Gourmet Shop,C_Greek Restaurant,C_Grocery Store,C_Gym,C_Gym / Fitness Center,C_Gym Pool,C_Harbor / Marina,C_Hardware Store,C_Hawaiian Restaurant,C_Health & Beauty Service,C_Historic Site,C_History Museum,C_Hobby Shop,C_Hockey Arena,C_Hockey Rink,C_Home Service,C_Hookah Bar,C_Hospital,C_Hostel,C_Hot Dog Joint,C_Hot Spring,C_Hotel,C_Hotel Bar,C_Hotel Pool,C_IT Services,C_Ice Cream Shop,C_Indian Restaurant,C_Indie Movie Theater,C_Indie Theater,C_Indoor Play Area,C_Intersection,C_Irish Pub,C_Island,C_Israeli Restaurant,C_Italian Restaurant,C_Japanese Restaurant,C_Jazz Club,C_Jewelry Store,C_Jewish Restaurant,C_Juice Bar,C_Kebab Restaurant,C_Kitchen Supply Store,C_Korean Restaurant,C_Lake,C_Laser Tag,C_Latin American Restaurant,C_Laundry Service,C_Lebanese Restaurant,C_Library,C_Light Rail Station,C_Lighting Store,C_Lingerie Store,C_Liquor Store,C_Lounge,C_Market,C_Mediterranean Restaurant,C_Men's Store,C_Metro Station,C_Mexican Restaurant,C_Middle Eastern Restaurant,C_Mini Golf,C_Miscellaneous Shop,C_Mobile Phone Shop,C_Modern European Restaurant,C_Molecular Gastronomy Restaurant,C_Monastery,C_Monument / Landmark,C_Moroccan Restaurant,C_Motel,C_Motorcycle Shop,C_Mountain,C_Movie Theater,C_Moving Target,C_Multiplex,C_Museum,C_Music School,C_Music Venue,C_Nature Preserve,C_Neighborhood,C_Newsstand,C_Nightclub,C_Noodle House,C_North Indian Restaurant,C_Office,C_Opera House,C_Organic Grocery,C_Other Great Outdoors,C_Other Nightlife,C_Outdoor Sculpture,C_Outdoors & Recreation,C_Outlet Store,C_Palace,C_Paper / Office Supplies Store,C_Park,C_Parking,C_Pastry Shop,C_Pedestrian Plaza,C_Performing Arts Venue,C_Perfume Shop,C_Peruvian Restaurant,C_Pet Store,C_Pharmacy,C_Photography Studio,C_Piano Bar,C_Pier,C_Pizza Place,C_Planetarium,C_Platform,C_Playground,C_Plaza,C_Pool,C_Pool Hall,C_Portuguese Restaurant,C_Post Office,C_Pub,C_Public Art,C_RV Park,C_Racetrack,C_Ramen Restaurant,C_Recreation Center,C_Rental Car Location,C_Resort,C_Rest

,Town,C_Accessories Store,C_Airport,C_Airport Lounge,C_Airport Service,C_Airport Terminal,C_American Restaurant,C_Apres Ski Bar,C_Arcade,C_Argentinian Restaurant,C_Art Gallery,C_Art Museum,C_Arts & Crafts Store,C_Asian Restaurant,C_Athletics & Sports,C_Australian Restaurant,C_Auto Dealership,C_Auto Garage,C_Auto Workshop,C_BBQ Joint,C_Bagel Shop,C_Bakery,C_Bank,C_Bar,C_Basketball Stadium,C_Bath House,C_Bathing Area,C_Beach,C_Beach Bar,C_Bed & Breakfast,C_Beer Garden,C_Beer Store,C_Big Box Store,C_Bistro,C_Boat or Ferry,C_Bookstore,C_Border Crossing,C_Botanical Garden,C_Boutique,C_Bowling Alley,C_Bratwurst Joint,C_Brazilian Restaurant,C_Breakfast Spot,C_Brewery,C_Bridge,C_Bubble Tea Shop,C_Buffet,C_Burger Joint,C_Burrito Place,C_Bus Line,C_Bus Station,C_Bus Stop,C_Butcher,C_Cable Car,C_Cafeteria,C_Café,C_Campground,C_Canal Lock,C_Candy Store,C_Car Wash,C_Casino,C_Castle,C_Cheese Shop,C_Chinese Restaurant,C_Chocolate Shop,C_Christmas Market,C_Church,C_Circus,C_City,C_Climbing Gym,C_Clothing Store,C_Cocktail Bar,C_Coffee Shop,C_Comedy Club,C_Comfort Food Restaurant,C_Community Center,C_Concert Hall,C_Construction & Landscaping,C_Convenience Store,C_Convention Center,C_Cosmetics Shop,C_Coworking Space,C_Creperie,C_Cuban Restaurant,C_Cultural Center,C_Cupcake Shop,C_Dance Studio,C_Deli / Bodega,C_Department Store,C_Dessert Shop,C_Dim Sum Restaurant,C_Diner,C_Discount Store,C_Dive Bar,C_Doner Restaurant,C_Donut Shop,C_Drugstore,C_Eastern European Restaurant,C_Electronics Store,C_English Restaurant,C_Ethiopian Restaurant,C_Event Space,C_Exhibit,C_Factory,C_Falafel Restaurant,C_Farm,C_Farmers Market,C_Fast Food Restaurant,C_Field,C_Fish & Chips Shop,C_Fishing Spot,C_Flea Market,C_Flower Shop,C_Fondue Restaurant,C_Food,C_Food & Drink Shop,C_Food Court,C_Food Truck,C_Forest,C_Fountain,C_French Restaurant,C_Fried Chicken Joint,C_Furniture / Home Store,C_Garden,C_Garden Center,C_Gas Station,C_Gastropub,C_General College & University,C_General Entertainment,C_General Travel,C_German Restaurant,C_Gift Shop,C_Golf Course,C_Gourmet Shop,C_Greek Restaurant,C_Grocery Store,C_Gym,C_Gym / Fitness Center,C_Gym Pool,C_Harbor / Marina,C_Hardware Store,C_Hawaiian Restaurant,C_Health & Beauty Service,C_Historic Site,C_History Museum,C_Hobby Shop,C_Hockey Arena,C_Hockey Rink,C_Home Service,C_Hookah Bar,C_Hospital,C_Hostel,C_Hot Dog Joint,C_Hot Spring,C_Hotel,C_Hotel Bar,C_Hotel Pool,C_IT Services,C_Ice Cream Shop,C_Indian Restaurant,C_Indie Movie Theater,C_Indie Theater,C_Indoor Play Area,C_Intersection,C_Irish Pub,C_Island,C_Israeli Restaurant,C_Italian Restaurant,C_Japanese Restaurant,C_Jazz Club,C_Jewelry Store,C_Jewish Restaurant,C_Juice Bar,C_Kebab Restaurant,C_Kitchen Supply Store,C_Korean Restaurant,C_Lake,C_Laser Tag,C_Latin American Restaurant,C_Laundry Service,C_Lebanese Restaurant,C_Library,C_Light Rail Station,C_Lighting Store,C_Lingerie Store,C_Liquor Store,C_Lounge,C_Market,C_Mediterranean Restaurant,C_Men's Store,C_Metro Station,C_Mexican Restaurant,C_Middle Eastern Restaurant,C_Mini Golf,C_Miscellaneous Shop,C_Mobile Phone Shop,C_Modern European Restaurant,C_Molecular Gastronomy Restaurant,C_Monastery,C_Monument / Landmark,C_Moroccan Restaurant,C_Motel,C_Motorcycle Shop,C_Mountain,C_Movie Theater,C_Moving Target,C_Multiplex,C_Museum,C_Music School,C_Music Venue,C_Nature Preserve,C_Neighborhood,C_Newsstand,C_Nightclub,C_Noodle House,C_North Indian Restaurant,C_Office,C_Opera House,C_Organic Grocery,C_Other Great Outdoors,C_Other Nightlife,C_Outdoor Sculpture,C_Outdoors & Recreation,C_Outlet Store,C_Palace,C_Paper / Office Supplies Store,C_Park,C_Parking,C_Pastry Shop,C_Pedestrian Plaza,C_Performing Arts Venue,C_Perfume Shop,C_Peruvian Restaurant,C_Pet Store,C_Pharmacy,C_Photography Studio,C_Piano Bar,C_Pier,C_Pizza Place,C_Planetarium,C_Platform,C_Playground,C_Plaza,C_Pool,C_Pool Hall,C_Portuguese Restaurant,C_Post Office,C_Pub,C_Public Art,C_RV Park,C_Racetrack,C_Ramen Restaurant,C_Recreation Center,C_Rental Car Location,C_Resort,C_Rest